In [1]:
#load Spark
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext()
ss = SparkSession.builder.appName("project1").getOrCreate()

In [38]:
import pandas as pd
import boto3
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.feature import Imputer


In [3]:
#df1 = pd.read_csv("s3://502projectsong/cleaned-subset.csv")
#df1.head(5)

In [43]:
df=ss.read.load("s3://502projectsong/clean_big_subset.csv",format="csv",header="true")
#df.na.drop()

In [44]:
from pyspark.sql.types import FloatType,IntegerType
from pyspark.sql.functions import when

In [45]:
#drop meaning less columns
df = df.drop('_c0', 'artist_id', 'artist_location', 'artist_name', 'title', 'song_id', 'release')

In [46]:
# convert artist terms 
df = df.withColumn('artist_terms', length(df.artist_terms))

#extract value in paranthesis
df = df.withColumn("segments_loudness_max_time",regexp_extract(df['segments_loudness_max_time'], '[0-9]+',0))
df = df.withColumn("similar_artists",regexp_extract(df['similar_artists'], '[0-9]+',0))
split_col = split(df['segments_pitches'], ',')
df = df.withColumn('segments_pitches_1', split_col.getItem(0))
df = df.withColumn('segments_pitches', split_col.getItem(1))
df = df.withColumn("segments_pitches_1",regexp_extract(df['segments_pitches_1'], '[0-9]+',0))
df = df.withColumn("segments_pitches", substring_index(df.segments_pitches, ')', 1))

split_col = split(df['segments_timbre'], ',')
df = df.withColumn('segments_timbre_1', split_col.getItem(0))
df = df.withColumn('segments_timbre', split_col.getItem(1))
df = df.withColumn("segments_timbre_1",regexp_extract(df['segments_timbre_1'], '[0-9]+',0))
df = df.withColumn("segments_timbre", substring_index(df.segments_pitches, ')', 1))

df.show(1)

+--------------------+------------------+-----------------+---------------+----------------+------------+-----------------+---------------+-----------+----------------+-----------+------------+--------+--------------+------+---+--------------+--------+----+---------------+-------------------+--------------+-------------------+---------------------+--------------------------+-----------------------+----------------+--------------+---------------+---------------+---------------+-----------------+-----------------+------------+-------+--------------+-------------------------+----+------------------+-----------------+
|analysis_sample_rate|artist_familiarity|artist_hotttnesss|artist_latitude|artist_longitude|artist_terms|artist_terms_freq|bars_confidence| bars_start|beats_confidence|beats_start|danceability|duration|end_of_fade_in|energy|key|key_confidence|loudness|mode|mode_confidence|sections_confidence|sections_start|segments_confidence|segments_loudness_max|segments_loudness_max_time|s

In [47]:
#df.show(5)
df=df.filter(df.song_hotttnesss.isNotNull())
df = df.withColumn('song_hotttnesss',when(df.song_hotttnesss <= 0.5, 0).otherwise(1))
df = df.dropna(how='any')

#convert field type  df.call_time.cast('float')
for item in df.columns:
    df = df.withColumn(item, df[item].cast('float'))

In [48]:
inputs = [i for i in df.schema.names if i != 'song_hotttnesss']

In [52]:
vectorAssembler = VectorAssembler(inputCols =inputs, outputCol = 'features')
v_df = vectorAssembler.transform(df)
v_df = v_df.select(['features', 'song_hotttnesss'])
v_df.show(3)

+--------------------+---------------+
|            features|song_hotttnesss|
+--------------------+---------------+
|[22050.0,0.647933...|            0.0|
|[22050.0,0.607532...|            0.0|
|[22050.0,0.621432...|            0.0|
+--------------------+---------------+
only showing top 3 rows



In [54]:
# Scalerizer = StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
# v_df = Scalerizer.fit(v_df).transform(v_df)
# v_df.show(3)

In [55]:
splits = v_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [57]:
#lr = LinearRegression(featuresCol = 'Scaled_features', labelCol='song_hotttnesss', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr = LogisticRegression(featuresCol = 'features', labelCol='song_hotttnesss', maxIter=10, regParam=0.3, elasticNetParam=0.8)

lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Py4JJavaError: An error occurred while calling o1706.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 17.0 failed 4 times, most recent failure: Lost task 2.3 in stage 17.0 (TID 94, ip-172-31-83-179.ec2.internal, executor 16): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<analysis_sample_rate_double_VectorAssembler_fe2a74aed5b4:double,artist_familiarity_double_VectorAssembler_fe2a74aed5b4:double,artist_hotttnesss_double_VectorAssembler_fe2a74aed5b4:double,artist_latitude_double_VectorAssembler_fe2a74aed5b4:double,artist_longitude_double_VectorAssembler_fe2a74aed5b4:double,artist_terms_double_VectorAssembler_fe2a74aed5b4:double,artist_terms_freq_double_VectorAssembler_fe2a74aed5b4:double,bars_confidence_double_VectorAssembler_fe2a74aed5b4:double,bars_start_double_VectorAssembler_fe2a74aed5b4:double,beats_confidence_double_VectorAssembler_fe2a74aed5b4:double,beats_start_double_VectorAssembler_fe2a74aed

In [14]:
trainingSummary = lr_model.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

objectiveHistory:
0.6766468575794772
+---+---+
|FPR|TPR|
+---+---+
|0.0|0.0|
|1.0|1.0|
|1.0|1.0|
+---+---+

areaUnderROC: 0.5


In [15]:
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Accuracy: 0.5905797101449275
FPR: 0.5905797101449275
TPR: 0.5905797101449275
F-measure: 0.43856260935591423
Precision: 0.3487843940348666
Recall: 0.5905797101449275


In [16]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","song_hotttnesss","Scaled_features").show(5)

+----------+---------------+--------------------+
|prediction|song_hotttnesss|     Scaled_features|
+----------+---------------+--------------------+
|       0.0|              1|[1.25329583464309...|
|       0.0|              0|[2.09470085287905...|
|       0.0|              0|[2.70782201816297...|
|       0.0|              0|[2.94231306058092...|
|       0.0|              0|[3.05605016524056...|
+----------+---------------+--------------------+
only showing top 5 rows



In [18]:
#test_result = lr_model.evaluate(test_df)
#print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

evaluator = BinaryClassificationEvaluator( labelCol = 'song_hotttnesss')
print('Test Area Under ROC', evaluator.evaluate(lr_predictions))


Test Area Under ROC 0.5


In [20]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))

numIterations: 1
objectiveHistory: [0.6766468575794772]


In [23]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'song_hotttnesss')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="song_hotttnesss", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = 0.459356


In [24]:
dt_model.featureImportances

SparseVector(40, {0: 0.5097, 1: 0.0629, 2: 0.0166, 3: 0.0613, 4: 0.0056, 6: 0.0677, 9: 0.0146, 11: 0.0226, 14: 0.0246, 17: 0.038, 18: 0.011, 21: 0.0011, 24: 0.0051, 35: 0.0227, 37: 0.0462, 38: 0.0903})